<a href="https://colab.research.google.com/github/aymaw/all/blob/main/%20gma_score_prediction_from_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part A

In [ ]:
# Mount Google Drive and create directories
from google.colab import drive
import os
from IPython.display import Video

drive.mount('/content/drive')

video_dir = "/content/drive/MyDrive/mypose_videos"
output_dir = "/content/drive/MyDrive/mypose_outputs"
os.makedirs(video_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Install Dependencies
!sudo apt update
!sudo apt install python3.8-full python3-pip -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

!pip install openmim gdown ipykernel
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!mim install mmengine
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html
!mim install "mmpose>=1.1.0"

!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install -r requirements.txt
!pip install -v -e .

# Select Models for Testing
models = ['vitpose-h', 'hrnet-w48', 'resnet50', 'openpose']

# Test 4 Models
for model in models:
    print(f"Testing model: {model}")
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        vis_output = f"{output_dir}/{model}_{video_file.split('.')[0]}_vis.mp4"
        pred_output = f"{output_dir}/{model}_{video_file.split('.')[0]}_pred.json"

        !python ./demo/inferencer_demo.py {video_path} \
            --pose2d {model} --pred-out-dir {pred_output} \
            --vis-out-dir {vis_output} --skeleton-style openpose \
            --radius 5 --thickness 3

        # Display video output
        display(Video(vis_output))

Part B

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

# Step 1: Ensure the utility files are downloaded and accessible
# Download necessary files if they don't already exist
utility_files = {
    "kinematics.py": "https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/main/utils/kinematics.py",
    "processing.py": "https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/main/utils/processing.py",
    "skeleton.py": "https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/main/utils/skeleton.py",
    "circstat.py": "https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/main/utils/circstat.py",
}

for filename, url in utility_files.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        !wget -q -O {filename} {url}
    else:
        print(f"{filename} already exists, skipping download.")

# Step 2: Update the system path to include the current directory
sys.path.append(os.getcwd())

# Step 3: Import utility functions
try:
    from kinematics import compute_displacement_velocity, compute_angles
    from processing import interpolate_data, smooth_data
    from skeleton import normalize_skeleton
    import circstat as CS
except ModuleNotFoundError as e:
    print("Error importing utility files. Ensure all files are downloaded and paths are set correctly.")
    raise e

# Step 4: Define paths
video_dir = "/content/drive/MyDrive/mypose_videos"  # Update with your video folder path
output_dir = "/content/drive/MyDrive/mypose_outputs"  # Update with your desired output folder path
os.makedirs(output_dir, exist_ok=True)

# Step 5: Load videos and process data
# Example: Process pose data from videos in `video_dir` and save outputs in `output_dir`
def process_videos(video_dir, output_dir):
    # List all videos in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]

    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(video_dir, video_file)
        output_path = os.path.join(output_dir, os.path.splitext(video_file)[0] + "_features.json")

        try:
            # Simulate loading video data and processing it
            print(f"Processing video: {video_file}")

            # Placeholder for actual pose extraction (replace with real implementation)
            # Example: Call pose extraction model
            skeleton_data = extract_pose_data(video_path)  # Replace with your pose extraction function

            # Normalize the skeleton
            normalized_skeleton = normalize_skeleton(skeleton_data)

            # Interpolate and smooth the skeleton data
            interpolated_skeleton = interpolate_data(normalized_skeleton)
            smoothed_skeleton = smooth_data(interpolated_skeleton)

            # Compute displacement velocity and angles
            displacement_velocity = compute_displacement_velocity(smoothed_skeleton)
            joint_angles = compute_angles(smoothed_skeleton)

            # Combine features into a dictionary
            features = {
                "displacement_velocity": displacement_velocity,
                "joint_angles": joint_angles,
            }

            # Save features to a JSON file
            with open(output_path, "w") as json_file:
                json.dump(features, json_file, indent=4)
            print(f"Features saved to {output_path}")

        except Exception as e:
            print(f"Error processing video {video_file}: {e}")

# Placeholder for pose extraction function
def extract_pose_data(video_path):
    """
    Simulates pose extraction from a video file.
    Replace with your actual pose extraction function or library call.
    """
    # TODO: Add actual implementation
    print(f"Extracting pose data from {video_path}...")
    return [{"x": 0, "y": 0}]  # Dummy skeleton data

# Run the processing pipeline
process_videos(video_dir, output_dir)


Part C

In [ ]:
# Install Necessary Libraries (uncomment if running for the first time)
# !pip install --upgrade pip setuptools wheel
# !apt-get install swig build-essential libssl-dev libffi-dev python3-dev
# !pip install auto-sklearn

# Import Required Libraries
import pandas as pd
import numpy as np
from autosklearn.classification import AutoSklearnClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import json

# Step 1: Load the Pose Data
pose_data_path = "/content/drive/MyDrive/pose_data.json"

# Loading a JSON file with pose data
with open(pose_data_path, 'r') as file:
    pose_data = json.load(file)

# Convert JSON to DataFrame
pose_df = pd.json_normalize(pose_data)
print(f"Loaded Data Shape: {pose_df.shape}")
print(pose_df.head())

# Step 2: Preprocess Data
# Drop non-numeric or irrelevant columns
irrelevant_columns = ['frame_id', 'timestamp']
pose_df = pose_df.drop(columns=irrelevant_columns, errors='ignore')

# Handle missing values
pose_df = pose_df.fillna(method='ffill').fillna(method='bfill')

# Normalize numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pose_df.iloc[:, :] = scaler.fit_transform(pose_df.iloc[:, :])

# Step 3: Split Data into Features (X) and Labels (y)
X = pose_df.drop(columns=['label'], errors='ignore')
y = pose_df['label']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train Auto-Sklearn Classifier
model = AutoSklearnClassifier(
    time_left_for_this_task=300,  # Total time for model optimization in seconds
    per_run_time_limit=30,       # Time limit for each model training
    memory_limit=1024            # Memory limit in MB
)

print("Training AutoSklearnClassifier...")
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
print("Evaluating the model...")
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class="ovr")

print(f"Accuracy: {accuracy}")
print(f"ROC-AUC Score: {roc_auc}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Step 6: Save the Model (Optional)
import joblib
model_save_path = "/content/auto_sklearn_model.pkl"
joblib.dump(model, model_save_path)
print(f"Model saved to {model_save_path}")
